# What's this PyTorch business?

You've written a lot of code in this assignment to provide a whole host of neural network functionality. 
Dropout and 2D convolutions are some of the workhorses of deep learning in computer vision. You've also worked hard to make your code efficient and vectorized.

For the last part of this assignment, though, we're going to leave behind your beautiful codebase and 
instead migrate to one of the popular deep learning frameworks, PyTorch.

### What is PyTorch?

PyTorch is a system for executing dynamic computational graphs over Tensor objects that behave similarly as numpy ndarray. It comes with a powerful automatic differentiation engine that removes the need for manual back-propagation. 

### Why?

* Our code will now run on GPUs! Much faster training. When using a framework like PyTorch or TensorFlow you can harness the power of the GPU for your own custom neural network architectures without having to write CUDA code directly (which is beyond the scope of this class).
* We want you to be ready to use one of these frameworks for your project so you can experiment more efficiently than if you were writing every feature you want to use by hand. 
* We want you to stand on the shoulders of giants! PyTorch is an excellent framework that will make your
 lives a lot easier, and now that you understand its gut, you are free to use it :) 
* We want you to be exposed to the sort of deep learning code you might run into in academia or industry.

### PyTorch versions
This notebook assumes that you are using **PyTorch version 1.0**. In some of the previous versions (e.g. before 0.4), Tensors had to be wrapped in Variable objects to be used in autograd; however Variables have now been deprecated. In addition 1.0 also separates a Tensor's datatype from its device, and uses numpy-style factories for constructing Tensors rather than directly invoking Tensor constructors.

## How will I learn PyTorch?

Justin Johnson has made an excellent [tutorial](https://github.com/jcjohnson/pytorch-examples) for PyTorch. 

You can also find the detailed [API doc](http://pytorch.org/docs/stable/index.html) here. If you have other questions that are not addressed by the API docs, the [PyTorch forum](https://discuss.pytorch.org/) is a much better place to ask than StackOverflow.

## Install PyTorch 1.0 (ONLY IF YOU ARE WORKING LOCALLY)

1. Have the latest version of Anaconda installed on your machine.
2. Create a new conda environment starting from Python 3.7. In this setup example, we'll call it `torch_env`.
3. Run the command: `conda activate torch_env`
4. Run the command: `pip install torch==1.0 torchvision==0.4.1`

## OR JUST WORK ON THIS NOTEBOOK ON GOOGLE COLAB (IF YOU DONT HAVE A LOCAL GPU)
https://colab.research.google.com/

https://www.youtube.com/watch?v=PitcORQSjNM This video summarizes how you can open a Colab, request and allocate a GPU for your work and import Tensorflow in addition. You can follow the steps to create your GPU backed environment in Colab and then just use below cells to import PyTorch.

# Table of Contents

This assignment has 5 parts. You will learn PyTorch on **three different levels of abstraction**, which will help you understand it better and prepare you for the final project. 

1. Part I, Preparation: we will use CIFAR-10 dataset.
2. Part II, Barebones PyTorch: **Abstraction level 1**, we will work directly with the lowest-level PyTorch Tensors. 
3. Part III, PyTorch Module API: **Abstraction level 2**, we will use `nn.Module` to define arbitrary neural network architecture. 
4. Part IV, PyTorch Sequential API: **Abstraction level 3**, we will use `nn.Sequential` to define a linear feed-forward network very conveniently. 
5. Part V, CIFAR-10 open-ended challenge: please implement your own network to get as high accuracy as possible on CIFAR-10. You can experiment with any layer, optimizer, hyperparameters or other advanced features. 

Here is a table of comparison:

| API           | Flexibility | Convenience |
|---------------|-------------|-------------|
| Barebone      | High        | Low         |
| `nn.Module`     | High        | Medium      |
| `nn.Sequential` | Low         | High        |

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer',
        'dog', 'frog', 'horse', 'ship', 'truck')
# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./comp451/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./comp451/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./comp451/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


# Part II. Barebones PyTorch

PyTorch ships with high-level APIs to help us define model architectures conveniently, which we will cover in Part II of this tutorial. In this section, we will start with the barebone PyTorch elements to understand the autograd engine better. After this exercise, you will come to appreciate the high-level model API more.

We will start with a simple fully-connected ReLU network with two hidden layers and no biases for CIFAR classification. 
This implementation computes the forward pass using operations on PyTorch Tensors, and uses PyTorch autograd to compute gradients. It is important that you understand every line, because you will write a harder version after the example.

When we create a PyTorch Tensor with `requires_grad=True`, then operations involving that Tensor will not just compute values; they will also build up a computational graph in the background, allowing us to easily backpropagate through the graph to compute gradients of some Tensors with respect to a downstream loss. Concretely if x is a Tensor with `x.requires_grad == True` then after backpropagation `x.grad` will be another Tensor holding the gradient of x with respect to the scalar loss at the end.

### PyTorch Tensors: Flatten Function
A PyTorch Tensor is conceptionally similar to a numpy array: it is an n-dimensional grid of numbers, and like numpy PyTorch provides many functions to efficiently operate on Tensors. As a simple example, we provide a `flatten` function below which reshapes image data for use in a fully-connected neural network.

Recall that image data is typically stored in a Tensor of shape N x C x H x W, where:

* N is the number of datapoints
* C is the number of channels
* H is the height of the intermediate feature map in pixels
* W is the width of the intermediate feature map in pixels

This is the right way to represent the data when we are doing something like a 2D convolution, that needs spatial understanding of where the intermediate features are relative to each other. When we use fully connected affine layers to process the image, however, we want each datapoint to be represented by a single vector -- it's no longer useful to segregate the different channels, rows, and columns of the data. So, we use a "flatten" operation to collapse the `C x H x W` values per representation into a single long vector. The flatten function below first reads in the N, C, H, and W values from a given batch of data, and then returns a "view" of that data. "View" is analogous to numpy's "reshape" method: it reshapes x's dimensions to be N x ??, where ?? is allowed to be anything (in this case, it will be C x H x W, but we don't need to specify that explicitly). 

In [4]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ')
    print(x)
    print('\nAfter flattening: ')
    print(flatten(x))

test_flatten()

Before flattening: 
tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])

After flattening: 
tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


### Barebones PyTorch: Three-Layer Network

Here we define a function `three_layer_fc` which performs the forward pass of a three-layer fully-connected ReLU network on a batch of image data. After defining the forward pass we check that it doesn't crash and that it produces outputs of the right shape by running zeros through the network.

You don't have to write any code here, but it's important that you read and understand the implementation.

In [5]:
import torch.nn.functional as F  # useful stateless functions (There are some cool functions)

def three_layer_fc(x, params):
    """
    A fully-connected neural networks; the architecture is:
    NN is fully connected -> ReLU -> fully connected layer -> ReLU -> fully connected layer.
    Note that this function only defines the forward pass; 
    PyTorch will take care of the backward pass for us.
    
    The input to the network will be a minibatch of data, of shape
    (N, d1, ..., dM) where d1 * ... * dM = D. The hidden layer will have H units,
    and the output layer will produce scores for C classes.
    
    Inputs:
    - x: A PyTorch Tensor of shape (N, d1, ..., dM) giving a minibatch of
      input data.
    - params: A list [w1, w2, w3] of PyTorch Tensors giving weights for the network;
      w1 has shape (D, H1) and w2 has shape (H1, H2) and w3 has shape (H2, C).
    
    Returns:
    - scores: A PyTorch Tensor of shape (N, C) giving classification scores for
      the input data x.
    """
    # first we flatten the image
    x = flatten(x)  # shape: [batch_size, C x H x W]
    
    w1, w2, w3 = params
    
    # Forward pass: compute predicted y using operations on Tensors. Since w1, w2 and
    # w3 have requires_grad=True, operations involving these Tensors will cause
    # PyTorch to build a computational graph, allowing automatic computation of
    # gradients. Since we are no longer implementing the backward pass by hand we
    # don't need to keep references to intermediate values.
    # you can also use `.clamp(min=0)`, equivalent to F.relu()
    
    x = F.relu(x.mm(w1)) #Performs matrix multiplication
    x = F.relu(x.mm(w2))
    x = x.mm(w3)
    return x
    

def three_layer_fc_test():
    hidden_layer_size = 42
    x = torch.zeros((64, 50), dtype=dtype)  # minibatch size 64, feature dimension 50
    w1 = torch.zeros((50, hidden_layer_size), dtype=dtype)
    w2 = torch.zeros((hidden_layer_size, hidden_layer_size), dtype=dtype)
    w3 = torch.zeros((hidden_layer_size, 10), dtype=dtype)
    scores = three_layer_fc(x, [w1, w2, w3])
    print(scores.size())  # you should see [64, 10]

three_layer_fc_test()

torch.Size([64, 10])


### Barebones PyTorch: Four-Layer ConvNet

Here you will complete the implementation of the function `four_layer_convnet`, which will perform the forward pass of a four-layer convolutional network. Like above, we can immediately test our implementation by passing zeros through the network. The network should have the following architecture:

1. A convolutional layer (with bias) with `channel_1` filters, each with shape `KW1 x KH1`, and zero-padding of two
2. LeakyReLU nonlinearity
3. A convolutional layer (with bias) with `channel_2` filters, each with shape `KW2 x KH2`, and zero-padding of one
4. LeakyReLU nonlinearity
5. A convolutional layer (with bias) with `channel_3` filters, each with shape `KW3 x KH3`, and zero-padding of one
6. LeakyReLU nonlinearity
7. Fully-connected layer with bias, producing scores for C classes.

Note that we have **no softmax activation** here after our fully-connected layer: this is because PyTorch's cross entropy loss performs a softmax activation for you, and by bundling that step in makes computation more efficient.

**HINT**: For convolutions: https://pytorch.org/docs/stable/nn.functional.html?highlight=torch%20nn%20functional%20conv2d#torch.nn.functional.conv2d; pay attention to the shapes of convolutional filters!

In [6]:
def four_layer_convnet(x, params, alpha=1e-2):
    """
    Performs the forward pass of a four-layer convolutional network with the
    architecture defined above.

    Inputs:
    - x: A PyTorch Tensor of shape (N, 3, H, W) giving a minibatch of images
    - params: A list of PyTorch Tensors giving the weights and biases for the
      network; should contain the following:
      - conv_w1: PyTorch Tensor of shape (channel_1, 3, KH1, KW1) giving weights
        for the first convolutional layer
      - conv_b1: PyTorch Tensor of shape (channel_1,) giving biases for the first
        convolutional layer
      - conv_w2: PyTorch Tensor of shape (channel_2, channel_1, KH2, KW2) giving
        weights for the second convolutional layer
      - conv_b2: PyTorch Tensor of shape (channel_2,) giving biases for the second
        convolutional layer
      - conv_w3: PyTorch Tensor of shape (channel_3, channel_2, KH3, KW3) giving
        weights for the third convolutional layer
      - conv_b3: PyTorch Tensor of shape (channel_3,) giving biases for the second
        convolutional layer
      - fc_w: PyTorch Tensor giving weights for the fully-connected layer. Can you
        figure out what the shape should be?
      - fc_b: PyTorch Tensor giving biases for the fully-connected layer. Can you
        figure out what the shape should be?
     - alpha: negative slop of Leaky ReLU layers
    
    Returns:
    - scores: PyTorch Tensor of shape (N, C) giving classification scores for x
    """
    conv_w1, conv_b1, conv_w2, conv_b2, conv_w3, conv_b3, fc_w, fc_b = params
    #print("Size of fc_w: ", fc_w.size())
    #print("Size of fc_b: ", fc_b.size())

    scores = None
    ################################################################################
    # TODO: Implement the forward pass for the three-layer ConvNet.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
    #print("Input size is: ", x.size())
    x = F.leaky_relu(F.conv2d(x,conv_w1,bias=conv_b1,padding=2),negative_slope=alpha)
    #print("First layer output size is: ", x.size())
    x = F.leaky_relu(F.conv2d(x,conv_w2,bias=conv_b2,padding=1),negative_slope=alpha)
    #print("Second layer output size is: ", x.size())
    x = F.leaky_relu(F.conv2d(x,conv_w3,bias=conv_b3,padding=1),negative_slope=alpha)
    #print("Third layer output size is: ", x.size())
    x = flatten(x)
    #print("Flatten size is: ", x.size())
    scores = x.mm(fc_w) + fc_b
    #print("Score size is: ", scores.size())
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################
    return scores

After defining the forward pass of the ConvNet above, run the following cell to test your implementation.

When you run this function, scores should have shape (64, 10).

In [7]:
def four_layer_convnet_test():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]

    conv_w1 = torch.zeros((6, 3, 5, 5), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b1 = torch.zeros((6,))  # out_channel
    conv_w2 = torch.zeros((9, 6, 3, 3), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b2 = torch.zeros((9,))  # out_channel
    conv_w3 = torch.zeros((12, 9, 3, 3), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b3 = torch.zeros((12,)) # out_channel

    # you must calculate the shape of the tensor after three conv layers, before the fully-connected layer
    fc_w = torch.zeros((12 * 32 * 32, 10))
    fc_b = torch.zeros(10)
    alpha = 5e-3

    scores = four_layer_convnet(x, [conv_w1, conv_b1, conv_w2, conv_b2, conv_w3, conv_b3, fc_w, fc_b], alpha)
    print(scores.size())  # you should see [64, 10]
four_layer_convnet_test()

torch.Size([64, 10])


### Barebones PyTorch: Initialization
Let's write a couple utility methods to initialize the weight matrices for our models.

- `random_weight(shape)` initializes a weight tensor with the Kaiming normalization method.
- `zero_weight(shape)` initializes a weight tensor with all zeros. Useful for instantiating bias parameters.

The `random_weight` function uses the Kaiming normal initialization method, described in:

He et al, *Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*, ICCV 2015, https://arxiv.org/abs/1502.01852

In [8]:
def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn is standard normal distribution generator. 
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# create a weight of shape [3 x 5]
# you should see the type `torch.cuda.FloatTensor` if you use GPU. 
# Otherwise it should be `torch.FloatTensor`
random_weight((3, 5))

tensor([[ 0.0746, -0.9196,  0.1266,  0.3416, -0.6513],
        [-1.3308,  0.9769,  0.0849, -0.5531, -1.0306],
        [-0.4221, -0.6016, -0.2416, -0.7714,  0.1752]], device='cuda:0',
       requires_grad=True)

### Barebones PyTorch: Check Accuracy
When training the model we will use the following function to check the accuracy of our model on the training or validation sets.

When checking accuracy we don't need to compute any gradients; as a result we don't need PyTorch to build a computational graph for us when we compute scores. To prevent a graph from being built we scope our computation under a `torch.no_grad()` context manager.

In [9]:
def check_accuracy_part2(loader, model_fn, params, alpha=None):
    """
    Check the accuracy of a classification model.
    
    Inputs:
    - loader: A DataLoader for the data split we want to check
    - model_fn: A function that performs the forward pass of the model,
      with the signature scores = model_fn(x, params)
    - params: List of PyTorch Tensors giving parameters of the model
    - alpha: optional, only used with LeakyReLUs
    
    Returns: Nothing, but prints the accuracy of the model
    """
    split = 'val' if loader.dataset.train else 'test'
    print('Checking accuracy on the %s set' % split)
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.int64)
            if alpha is not None:
                scores = model_fn(x, params, alpha)
            scores = model_fn(x, params)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

### BareBones PyTorch: Training Loop
We can now set up a basic training loop to train our network. We will train the model using stochastic gradient descent without momentum. We will use `torch.functional.cross_entropy` to compute the loss; you can [read about it here](http://pytorch.org/docs/stable/nn.html#cross-entropy).

The training loop takes as input the neural network function, a list of initialized parameters (`[w1, w2, w3]` in our example), and learning rate.

In [10]:
def train_part2(model_fn, params, learning_rate, alpha=None):
    """
    Train a model on CIFAR-10.
    
    Inputs:
    - model_fn: A Python function that performs the forward pass of the model.
      It should have the signature scores = model_fn(x, params) where x is a
      PyTorch Tensor of image data, params is a list of PyTorch Tensors giving
      model weights, and scores is a PyTorch Tensor of shape (N, C) giving
      scores for the elements in x.
    - params: List of PyTorch Tensors giving weights for the model
    - learning_rate: Python scalar giving the learning rate to use for SGD
    - alpha: negative slope of the Leaky ReLU. If other ReLU is used instead, this becomes None
    
    Returns: Nothing
    """
    for t, (x, y) in enumerate(loader_train):
        # Move the data to the proper device (GPU or CPU)
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        # Forward pass: compute scores and loss
        if alpha is not None:
            scores = model_fn(x, params, alpha)
        scores = model_fn(x, params)
        loss = F.cross_entropy(scores, y)

        # Backward pass: PyTorch figures out which Tensors in the computational
        # graph has requires_grad=True and uses backpropagation to compute the
        # gradient of the loss with respect to these Tensors, and stores the
        # gradients in the .grad attribute of each Tensor.
        loss.backward()

        # Update parameters. We don't want to backpropagate through the
        # parameter updates, so we scope the updates under a torch.no_grad()
        # context manager to prevent a computational graph from being built.
        with torch.no_grad():
            for w in params:
                w -= learning_rate * w.grad

                # Manually zero the gradients after running the backward pass
                w.grad.zero_()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part2(loader_val, model_fn, params, alpha)
            print()

### BareBones PyTorch: Train a Three-Layer Network
Now we are ready to run the training loop. We need to explicitly allocate tensors for the fully connected weights, `w1`, `w2` and `w3`. 

Each minibatch of CIFAR has 64 examples, so the tensor shape is `[64, 3, 32, 32]`. 

After flattening, `x` shape should be `[64, 3 * 32 * 32]`. This will be the size of the first dimension of `w1`. 
The second dimension of `w1` is the hidden layer size, which will also be the first dimension of `w2`. You can get the idea.

Finally, the output of the network is a 10-dimensional vector that represents the probability distribution over 10 classes. 

You don't need to tune any hyperparameters but you should see accuracies above 44% after training for one epoch.

In [11]:
hidden_layer_size = 2000
learning_rate = 1e-2

w1 = random_weight((3 * 32 * 32, hidden_layer_size))
w2 = random_weight((hidden_layer_size, hidden_layer_size))
w3 = random_weight((hidden_layer_size, 10))

train_part2(three_layer_fc, [w1, w2, w3], learning_rate)

Iteration 0, loss = 3.9110
Checking accuracy on the val set
Got 109 / 1000 correct (10.90%)

Iteration 100, loss = 1.8022
Checking accuracy on the val set
Got 332 / 1000 correct (33.20%)

Iteration 200, loss = 1.7935
Checking accuracy on the val set
Got 403 / 1000 correct (40.30%)

Iteration 300, loss = 1.5886
Checking accuracy on the val set
Got 399 / 1000 correct (39.90%)

Iteration 400, loss = 2.0985
Checking accuracy on the val set
Got 417 / 1000 correct (41.70%)

Iteration 500, loss = 1.6608
Checking accuracy on the val set
Got 437 / 1000 correct (43.70%)

Iteration 600, loss = 1.5633
Checking accuracy on the val set
Got 450 / 1000 correct (45.00%)

Iteration 700, loss = 1.5952
Checking accuracy on the val set
Got 456 / 1000 correct (45.60%)



### BareBones PyTorch: Training a ConvNet

In the below you should use the functions defined above to train a four-layer convolutional network on CIFAR. The network should have the following architecture:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. LeakyReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. LeakyReLU
5. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
6. LeakyReLU
7. Fully-connected layer (with bias) to compute scores for 10 classes

You should initialize your weight matrices using the `random_weight` function defined above, and you should initialize your bias vectors using the `zero_weight` function above.

You don't need to tune any hyperparameters, but if everything works correctly you should achieve an accuracy around 50% after one epoch.

In [12]:
learning_rate = 1e-2

channel_1 = 32
channel_2 = 16
channel_3 = 16
alpha = 1e-2

conv_w1 = None
conv_b1 = None
conv_w2 = None
conv_b2 = None
conv_w3 = None
conv_b3 = None
fc_w = None
fc_b = None

################################################################################
# TODO: Initialize the parameters of a three-layer ConvNet.                    #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

conv_w1 = random_weight([32,3,5,5])
conv_w2 = random_weight([16,32,3,3])
conv_w3 = random_weight([16,16,3,3])
conv_b1 = random_weight([32])
conv_b2 = random_weight([16])
conv_b3 = random_weight([16])
fc_w    = random_weight([16384,10]) 
fc_b    = random_weight([10])

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

params_four_layer = [conv_w1, conv_b1, conv_w2, conv_b2, conv_w3, conv_b3, fc_w, fc_b]
train_part2(four_layer_convnet, params_four_layer, learning_rate, alpha)

Iteration 0, loss = 3.9497
Checking accuracy on the val set
Got 79 / 1000 correct (7.90%)

Iteration 100, loss = 2.2369
Checking accuracy on the val set
Got 263 / 1000 correct (26.30%)

Iteration 200, loss = 1.7383
Checking accuracy on the val set
Got 366 / 1000 correct (36.60%)

Iteration 300, loss = 1.8349
Checking accuracy on the val set
Got 391 / 1000 correct (39.10%)

Iteration 400, loss = 1.7976
Checking accuracy on the val set
Got 430 / 1000 correct (43.00%)

Iteration 500, loss = 1.5634
Checking accuracy on the val set
Got 462 / 1000 correct (46.20%)

Iteration 600, loss = 1.7689
Checking accuracy on the val set
Got 375 / 1000 correct (37.50%)

Iteration 700, loss = 1.5699
Checking accuracy on the val set
Got 474 / 1000 correct (47.40%)



# Part III. PyTorch Module API

Barebone PyTorch requires that we track all the parameter tensors by hand. This is fine for small networks with a few tensors, but it would be extremely inconvenient and error-prone to track tens or hundreds of tensors in larger networks.

PyTorch provides the `nn.Module` API for you to define arbitrary network architectures, while tracking every learnable parameters for you. In Part II, we implemented SGD ourselves. PyTorch also provides the `torch.optim` package that implements all the common optimizers, such as RMSProp, Adagrad, and Adam. It even supports approximate second-order methods like L-BFGS! You can refer to the [doc](http://pytorch.org/docs/master/optim.html) for the exact specifications of each optimizer.

In addition, Module API enables us to implement tedious operations such as BatchNorm2D with extreme ease!

To use the Module API, follow the steps below:

1. Subclass `nn.Module`. Give your network class an intuitive name like `ThreeLayerFC`. 

2. In the constructor `__init__()`, define all the layers you need as class attributes. Layer objects like `nn.Linear` and `nn.Conv2d` are themselves `nn.Module` subclasses and contain learnable parameters, so that you don't have to instantiate the raw tensors yourself. `nn.Module` will track these internal parameters for you. Refer to the [doc](http://pytorch.org/docs/master/nn.html) to learn more about the dozens of builtin layers. **Warning**: don't forget to call the `super().__init__()` first!

3. In the `forward()` method, define the *connectivity* of your network. You should use the attributes defined in `__init__` as function calls that take tensor as input and output the "transformed" tensor. Do *not* create any new layers with learnable parameters in `forward()`! All of them must be declared upfront in `__init__`. 

After you define your Module subclass, you can instantiate it as an object and call it just like the NN forward function in part II.

### Module API: Three-Layer Network
Here is a concrete example of a 3-layer fully connected network:

In [13]:
class ThreeLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super().__init__()
        # assign layer objects to class attributes
        self.fc1 = nn.Linear(input_size, hidden_size1)
        # nn.init package contains convenient initialization methods
        # http://pytorch.org/docs/master/nn.html#torch-nn-init 
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        nn.init.kaiming_normal_(self.fc2.weight)
        self.fc3 = nn.Linear(hidden_size2, num_classes)
        nn.init.kaiming_normal_(self.fc3.weight)
    
    def forward(self, x):
        # forward always defines connectivity
        x = flatten(x)
        scores = self.fc3(F.relu(self.fc2(F.relu(self.fc1(x)))))
        return scores

def test_ThreeLayerFC():
    input_size = 50
    x = torch.zeros((64, input_size), dtype=dtype)  # minibatch size 64, feature dimension 50
    model = ThreeLayerFC(input_size, 42, 42, 10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_ThreeLayerFC()

torch.Size([64, 10])


### Module API: Four-Layer ConvNet
It's your turn to implement a 4-layer ConvNet followed by a fully connected layer. This time we will also include Spatial Batch Norm operations to our pipeline, thanks to PyTorch Modules such as nn.BatchNorm2d. The network architecture should be almost the same as in Part II, with the addition of BatchNorm2d layers:

1. Convolutional layer with `channel_1` 5x5 sfilters with zero-padding of 2
2. LeakyReLU
3. BatchNorm2d
4. Convolutional layer with `channel_2` 3x3 filters with zero-padding of 1
5. LeakyReLU
6. BatchNorm2d
7. Convolutional layer with `channel_3` 3x3 filters with zero-padding of 1
8. LeakyReLU
9. BatchNorm2d
10. Fully-connected layer to `num_classes` classes

You should initialize the weight matrices of convolutional layers using the Kaiming normal initialization method.

**HINT**: http://pytorch.org/docs/stable/nn.html#conv2d

https://pytorch.org/docs/stable/nn.init.html

https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html

After you implement the four-layer ConvNet, the `test_FourLayerConvNet` function will run your implementation; it should print `(64, 10)` for the shape of the output scores.

In [14]:
class FourLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, channel_3, num_classes, alpha):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a four-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        #Define overall architecture 

        self.c1       = nn.Conv2d(in_channel,channel_1,(5,5),padding=2)
        self.lrelu    = nn.LeakyReLU(negative_slope=alpha)
        self.bnorm2d1 = nn.BatchNorm2d(channel_1)
        
        self.c2 = nn.Conv2d(channel_1,channel_2,(3,3),padding=1)
        #nn,.LeakyReLU()
        self.bnorm2d2 = nn.BatchNorm2d(channel_2)
        
        self.c3 = nn.Conv2d(channel_2,channel_3,(3,3),padding=1)
        #nn.LeakyReLU()
        self.bnorm2d3 = nn.BatchNorm2d(channel_3)

        self.fc1 = nn.Linear(8192,num_classes)
        
        #Initialize weights
        nn.init.kaiming_normal_(self.c1.weight)
        nn.init.kaiming_normal_(self.c2.weight)
        nn.init.kaiming_normal_(self.c3.weight)
        nn.init.kaiming_normal_(self.fc1.weight)
        
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        #print("Input size is: ", x.size())
        x = self.bnorm2d1(self.lrelu(self.c1(x)))
        #print("First layer size is: ", x.size())
        x = self.bnorm2d2(self.lrelu(self.c2(x)))
        #print("Second layer size is: ", x.size())
        x = self.bnorm2d3(self.lrelu(self.c3(x)))
        #print("Third layer size is: ", x.size())
        x = flatten(x)
        #print("Flatted size is: ", x.size())
        scores = self.fc1(x)
    
    
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores


def test_FourLayerConvNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model = FourLayerConvNet(in_channel=3, channel_1=12, channel_2=8, channel_3=8, num_classes=10, alpha=1e-2)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_FourLayerConvNet()



torch.Size([64, 10])


### Module API: Check Accuracy
Given the validation or test set, we can check the classification accuracy of a neural network. 

This version is slightly different from the one in part II. You don't manually pass in the parameters anymore.

In [15]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [16]:
def check_accuracy_part35(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        return 100*acc

### Module API: Training Loop
We also use a slightly different training loop. Rather than updating the values of the weights ourselves, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [17]:
def train_part35(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    print_every = 200
    for e in range(epochs):
        print("Current epoch is: ", e)
        for t, (x, y) in enumerate(loader_train): #One minibatch which is 64
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                val_acc   = check_accuracy_part35(loader_val, model)
                train_acc = check_accuracy_part35(loader_train, model)
                print(train_acc,val_acc)
                # ...log the running loss
                writer.add_scalar('training loss',loss.item(),e * len(loader_train) + t)
                writer.add_scalar('training acc',train_acc, e * len(loader_train) + t)
                writer.add_scalar('validation acc',val_acc,e * len(loader_train) + t)                
                writer.add_scalar('train-val acc',train_acc-val_acc,e * len(loader_train) + t)                

                # ...log a Matplotlib Figure showing the model's predictions on a
                # random mini-batch
                writer.add_figure('predictions vs. actuals',
                                plot_classes_preds(model, x, y),
                                global_step=e * len(loader_train) + t)

                writer.flush()
                


In [18]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

### Module API: Train a Three-Layer Network
Now we are ready to run the training loop. In contrast to part II, we don't explicitly allocate parameter tensors anymore.

Simply pass the input size, hidden layer sizes, and number of classes (i.e. output size) to the constructor of `ThreeLayerFC`. 

You also need to define an optimizer that tracks all the learnable parameters inside `ThreeLayerFC`.

You don't need to tune any hyperparameters, but you should see model accuracies above 44% after training for one epoch.

In [19]:
hidden_layer_size = 2000
learning_rate = 1e-2
model = ThreeLayerFC(3 * 32 * 32, hidden_layer_size, hidden_layer_size, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_part34(model, optimizer)

Iteration 0, loss = 3.2263
Checking accuracy on validation set
Got 169 / 1000 correct (16.90)

Iteration 100, loss = 1.9944
Checking accuracy on validation set
Got 360 / 1000 correct (36.00)

Iteration 200, loss = 1.7625
Checking accuracy on validation set
Got 365 / 1000 correct (36.50)

Iteration 300, loss = 2.0250
Checking accuracy on validation set
Got 401 / 1000 correct (40.10)

Iteration 400, loss = 1.4427
Checking accuracy on validation set
Got 434 / 1000 correct (43.40)

Iteration 500, loss = 1.5845
Checking accuracy on validation set
Got 432 / 1000 correct (43.20)

Iteration 600, loss = 1.6098
Checking accuracy on validation set
Got 467 / 1000 correct (46.70)

Iteration 700, loss = 1.6717
Checking accuracy on validation set
Got 453 / 1000 correct (45.30)



### Module API: Train a Four-Layer ConvNet
You should now use the Module API to train a four-layer ConvNet on CIFAR. This should look very similar to training the three-layer network! You don't need to tune any hyperparameters, but you should achieve above above 50% after training for one epoch.

You should train the model using stochastic gradient descent without momentum.

In [20]:
learning_rate = 1e-2
channel_1 = 32
channel_2 = 16
channel_3 = 16
alpha = 1e-2

model = None
optimizer = None
################################################################################
# TODO: Instantiate your ThreeLayerConvNet model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

learning_rate     = 1e-2
model             = FourLayerConvNet(in_channel=3, channel_1=12, channel_2=8, channel_3=8, num_classes=10, alpha=1e-2)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 3.0239
Checking accuracy on validation set
Got 133 / 1000 correct (13.30)

Iteration 100, loss = 1.6936
Checking accuracy on validation set
Got 383 / 1000 correct (38.30)

Iteration 200, loss = 1.7873
Checking accuracy on validation set
Got 463 / 1000 correct (46.30)

Iteration 300, loss = 1.5469
Checking accuracy on validation set
Got 475 / 1000 correct (47.50)

Iteration 400, loss = 1.3113
Checking accuracy on validation set
Got 496 / 1000 correct (49.60)

Iteration 500, loss = 1.4609
Checking accuracy on validation set
Got 486 / 1000 correct (48.60)

Iteration 600, loss = 1.3149
Checking accuracy on validation set
Got 495 / 1000 correct (49.50)

Iteration 700, loss = 1.4950
Checking accuracy on validation set
Got 492 / 1000 correct (49.20)



# Part IV. PyTorch Sequential API

Part III introduced the PyTorch Module API, which allows you to define arbitrary learnable layers and their connectivity. 

For simple models like a stack of feed forward layers, you still need to go through 3 steps: subclass `nn.Module`, assign layers to class attributes in `__init__`, and call each layer one by one in `forward()`. Is there a more convenient way? 

Fortunately, PyTorch provides a container Module called `nn.Sequential`, which merges the above steps into one. It is not as flexible as `nn.Module`, because you cannot specify more complex topology than a feed-forward stack, but it's good enough for many use cases.

### Sequential API: Three-Layer Network
Let's see how to rewrite our three-layer fully connected network example with `nn.Sequential`, and train it using the training loop defined above.

Again, you don't need to tune any hyperparameters here, but you shoud achieve above 46% accuracy after one epoch of training.

In [21]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

hidden_layer_size = 2000
learning_rate = 1e-2

model = nn.Sequential(
    Flatten(),
    nn.Linear(3 * 32 * 32, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 10),
)

# you can use Nesterov momentum in optim.SGD
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model, optimizer)

Iteration 0, loss = 2.3156
Checking accuracy on validation set
Got 140 / 1000 correct (14.00)

Iteration 100, loss = 1.7584
Checking accuracy on validation set
Got 398 / 1000 correct (39.80)

Iteration 200, loss = 1.7294
Checking accuracy on validation set
Got 443 / 1000 correct (44.30)

Iteration 300, loss = 1.5309
Checking accuracy on validation set
Got 443 / 1000 correct (44.30)

Iteration 400, loss = 1.5575
Checking accuracy on validation set
Got 466 / 1000 correct (46.60)

Iteration 500, loss = 1.6289
Checking accuracy on validation set
Got 482 / 1000 correct (48.20)

Iteration 600, loss = 1.5828
Checking accuracy on validation set
Got 487 / 1000 correct (48.70)

Iteration 700, loss = 1.5634
Checking accuracy on validation set
Got 481 / 1000 correct (48.10)



### Sequential API: Four-Layer ConvNet
Here you should use `nn.Sequential` to define and train a four-layer ConvNet with the same architecture we used in Part III:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. LeakyReLU
3. BatchNorm2d
4. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
5. LeakyReLU
6. BatchNorm2d
7. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
8. LeakyReLU
9. BatchNorm2d
10. Fully-connected layer (with bias) to compute scores for 10 classes

You should initialize your weight matrices using the `random_weight` function defined above, and you should initialize your bias vectors using the `zero_weight` function above.

You should optimize your model using stochastic gradient descent with Nesterov momentum 0.9.

Again, you don't need to tune any hyperparameters but you should see accuracy above 50% after one epoch of training.

In [32]:
channel_1 = 32
channel_2 = 16
channel_3 = 16
learning_rate = 1e-2
alpha = 1e-2
seq_model = None
seq_optimizer = None
################################################################################
# TODO: Rewrite the 4-layer ConvNet with bias from Part III with the           #
# Sequential API. I.e., define seq_model.                                      #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


def init_weights(seq_layer):
    if type(seq_layer) == nn.Conv2d: 
        #print(seq_layer.weight)
        weight_shape = [seq_layer.weight.shape[1],seq_layer.weight.shape[0],seq_layer.weight.shape[2],seq_layer.weight.shape[3]]
        weight  = random_weight(weight_shape)
        weight  = torch.transpose(weight, 0, 1)
        seq_layer.weight = torch.nn.Parameter(weight)
        seq_layer.bias   = torch.nn.Parameter(zero_weight(seq_layer.bias.shape))
    
    if type(seq_layer) == nn.Linear:
        weight_shape = [seq_layer.weight.shape[1],seq_layer.weight.shape[0]]
        weight = random_weight(weight_shape)
        weight  = torch.transpose(weight, 0, 1)
        seq_layer.weight = torch.nn.Parameter(weight)
        seq_layer.bias   = torch.nn.Parameter(zero_weight(seq_layer.bias.shape))
        
        
seq_model = nn.Sequential(
    nn.Conv2d(3,channel_1,(5,5),padding=2),
    nn.LeakyReLU(negative_slope=alpha),
    nn.BatchNorm2d(channel_1),
    
    nn.Conv2d(channel_1,channel_2,(3,3),padding=1),
    nn.LeakyReLU(negative_slope=alpha),
    nn.BatchNorm2d(channel_2),
    
    nn.Conv2d(channel_2,channel_3,(3,3),padding=1),
    nn.LeakyReLU(negative_slope=alpha),
    nn.BatchNorm2d(channel_3),
    
    Flatten(),
    nn.Linear(16384,10),
)

seq_model.apply(init_weights)




# you can use Nesterov momentum in optim.SGD
seq_optimizer = optim.SGD(seq_model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(seq_model, seq_optimizer)

Iteration 0, loss = 2.9721
Checking accuracy on validation set
Got 174 / 1000 correct (17.40)

Iteration 100, loss = 1.6345
Checking accuracy on validation set
Got 428 / 1000 correct (42.80)

Iteration 200, loss = 1.5004
Checking accuracy on validation set
Got 458 / 1000 correct (45.80)

Iteration 300, loss = 1.4014
Checking accuracy on validation set
Got 486 / 1000 correct (48.60)

Iteration 400, loss = 1.4871
Checking accuracy on validation set
Got 504 / 1000 correct (50.40)

Iteration 500, loss = 1.3443
Checking accuracy on validation set
Got 519 / 1000 correct (51.90)

Iteration 600, loss = 1.3201
Checking accuracy on validation set
Got 549 / 1000 correct (54.90)

Iteration 700, loss = 1.1060
Checking accuracy on validation set
Got 569 / 1000 correct (56.90)



# Part V. CIFAR-10 open-ended challenge

In this section, you can experiment with whatever ConvNet architecture you'd like on CIFAR-10. 

Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves **at least 70%** accuracy on the CIFAR-10 **validation** set within 10 epochs. You can use the check_accuracy and train functions from above. You can use either `nn.Module` or `nn.Sequential` API. 

Describe what you did at the end of this notebook.

Here are the official API documentation for each component. One note: what we call in the class "spatial batch norm" is called "BatchNorm2D" in PyTorch.

* Layers in torch.nn package: http://pytorch.org/docs/stable/nn.html
* Activations: http://pytorch.org/docs/stable/nn.html#non-linear-activations
* Loss functions: http://pytorch.org/docs/stable/nn.html#loss-functions
* Optimizers: http://pytorch.org/docs/stable/optim.html


### Things you might try:
- **Filter size**: Above we used 5x5; would smaller filters be more efficient?
- **Number of filters**: Above we used 32 filters. Do more or fewer do better?
- **Pooling vs Strided Convolution**: Do you use max pooling or just stride convolutions?
- **Batch normalization**: Try adding spatial batch normalization after convolution layers and vanilla batch normalization after affine layers. Do your networks train faster?
- **Network architecture**: The network above has two layers of trainable parameters. Can you do better with a deep network? Good architectures to try include:
    - [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [batchnorm-relu-conv]xN -> [affine]xM -> [softmax or SVM]
- **Global Average Pooling**: Instead of flattening and then having multiple affine layers, perform convolutions until your image gets small (7x7 or so) and then perform an average pooling operation to get to a 1x1 image picture (1, 1 , Filter#), which is then reshaped into a (Filter#) vector. This is used in [Google's Inception Network](https://arxiv.org/abs/1512.00567) (See Table 1 for their architecture).
- **Regularization**: Add l2 weight regularization, or perhaps use Dropout.

### Tips for training
For each network architecture that you try, you should tune the learning rate and other hyperparameters. When doing this there are a couple important things to keep in mind:

- If the parameters are working well, you should see improvement within a few hundred iterations
- Remember the coarse-to-fine approach for hyperparameter tuning: start by testing a large range of hyperparameters for just a few training iterations to find the combinations of parameters that are working at all.
- Once you have found some sets of parameters that seem to work, search more finely around these parameters. You may need to train for more epochs.
- You should use the validation set for hyperparameter search, and save your test set for evaluating your architecture on the best parameters as selected by the validation set.

### Going above and beyond
If you are feeling adventurous there are many other features you can implement to try and improve your performance. You are **not required** to implement any of these, but don't miss the fun if you have time!

- Alternative optimizers: you can try Adam, Adagrad, RMSprop, etc.
- Alternative activation functions such as leaky ReLU, parametric ReLU, ELU, or MaxOut.
- Model ensembles
- Data augmentation
- New Architectures
  - [ResNets](https://arxiv.org/abs/1512.03385) where the input from the previous layer is added to the output.
  - [DenseNets](https://arxiv.org/abs/1608.06993) where inputs into previous layers are concatenated together.
  - [This blog has an in-depth overview](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

### Have fun and happy training! 

In [23]:
#Architecture
class winner(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, channel_3, num_classes, alpha):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a four-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        #Define overall architecture 

        self.c1       = nn.Conv2d(in_channel,channel_1,(3,3),padding=1)
        self.lrelu    = nn.LeakyReLU(negative_slope=alpha)
        #self.lrelu    = nn.PReLU()

        #self.lrelu    = nn.SELU()
        self.bnorm2d1 = nn.BatchNorm2d(channel_1)
        
        self.c2 = nn.Conv2d(channel_1,channel_2,(3,3),padding=1)
        #nn,.LeakyReLU()
        self.bnorm2d2 = nn.BatchNorm2d(channel_2)
        
        self.c3 = nn.Conv2d(channel_2,channel_3,(3,3),padding=1)
        self.c4 = nn.Conv2d(channel_3,channel_3,(3,3),padding=1)

        #nn.LeakyReLU()
        self.bnorm2d3 = nn.BatchNorm2d(channel_3)

        self.fc1 = nn.Linear(9000,num_classes)
        
        self.max_pool = nn.MaxPool2d(3, stride=2)
        
        #Initialize weights
        nn.init.kaiming_normal_(self.c1.weight)
        nn.init.kaiming_normal_(self.c2.weight)
        nn.init.kaiming_normal_(self.c3.weight)
        nn.init.kaiming_normal_(self.c4.weight)
        nn.init.kaiming_normal_(self.fc1.weight)
        
        self.dropout_conv = nn.Dropout(p=0.2)
        self.dropout_fc = nn.Dropout(p=0.5)
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        #print("Input size is: ", x.size())
        x = self.bnorm2d1(self.lrelu(self.c1(x)))
        #print("First layer size is: ", x.size())
        x = self.bnorm2d2(self.lrelu(self.c2(x)))
        #print("Second layer size is: ", x.size())
        
        x = self.max_pool(x)
        
        x = self.bnorm2d3(self.lrelu(self.c3(x)))

        #print("Third layer size is: ", x.size())
        x = flatten(x)
        #print("Flatted size is: ", x.size())
        scores = self.fc1(x)
    
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores


def test_cnn_challenge_model():
    x      = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model  =winner(in_channel=3, channel_1=60, channel_2=40, channel_3=40, num_classes=10, alpha=1e-2)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]

test_cnn_challenge_model()



torch.Size([64, 10])


In [24]:
#Architecture
class my_model(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, channel_3, num_classes, alpha):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a four-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        #Define overall architecture 

        self.c1       = nn.Conv2d(in_channel,channel_1,(3,3),padding=1)
        self.lrelu    = nn.LeakyReLU(negative_slope=alpha)
        #self.lrelu    = nn.PReLU()

        #self.lrelu    = nn.SELU()
        self.bnorm2d1 = nn.BatchNorm2d(channel_1)
        
        self.c2 = nn.Conv2d(channel_1,channel_2,(3,3),padding=1)
        #nn,.LeakyReLU()
        self.bnorm2d2 = nn.BatchNorm2d(channel_2)
        
        self.c3 = nn.Conv2d(channel_2,channel_3,(3,3),padding=1)
        self.c4 = nn.Conv2d(channel_3,channel_3,(3,3),padding=1)

        #nn.LeakyReLU()
        self.bnorm2d3 = nn.BatchNorm2d(channel_3)

        self.fc1 = nn.Linear(9000,num_classes)
        
        self.max_pool = nn.MaxPool2d(3, stride=2)
        
        #Initialize weights
        nn.init.kaiming_normal_(self.c1.weight)
        nn.init.kaiming_normal_(self.c2.weight)
        nn.init.kaiming_normal_(self.c3.weight)
        nn.init.kaiming_normal_(self.c4.weight)
        nn.init.kaiming_normal_(self.fc1.weight)
        
        self.dropout_conv = nn.Dropout(p=0.2)
        self.dropout_fc = nn.Dropout(p=0.5)
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        #print("Input size is: ", x.size())
        x = self.dropout_conv(self.bnorm2d1(self.lrelu(self.c1(x))))
        #print("First layer size is: ", x.size())
        x = self.dropout_conv(self.bnorm2d2(self.lrelu(self.c2(x))))
        #print("Second layer size is: ", x.size())
        
        x = self.max_pool(x)
        
        x = self.dropout_conv(self.bnorm2d3(self.lrelu(self.c3(x))))
        x = self.dropout_conv(self.bnorm2d3(self.lrelu(self.c4(x))))

        #print("Third layer size is: ", x.size())
        x = flatten(x)
        #print("Flatted size is: ", x.size())
        scores = self.dropout_fc(self.fc1(x)) 
    
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores


def test_cnn_challenge_model():
    x      = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model  =my_model(in_channel=3, channel_1=60, channel_2=40, channel_3=40, num_classes=10, alpha=1e-2)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]

test_cnn_challenge_model()



torch.Size([64, 10])


In [31]:
################################################################################
# TODO:                                                                        #         
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
################################################################################
cnn_challenge_model = None
cnn_challenge_optimizer = None

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

#cnn_challenge_model     = winner(in_channel=3, channel_1=12, channel_2=8, channel_3=8, num_classes=10, alpha=1e-2)
#cnn_challenge_optimizer = optim.AdamW(cnn_challenge_model.parameters())



# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

# You should get at least 70% accuracy
#train_part35(cnn_challenge_model, cnn_challenge_optimizer, epochs=10)

'\nIts on the below\n'

In [26]:
import matplotlib.pyplot as plt

def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.cpu().numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [27]:
# helper function
def select_n_random(data, labels, n=100):
    '''
    Selects n random datapoints and their corresponding labels from a dataset
    '''
    assert len(data) == len(labels)

    perm  = torch.randperm(len(data))
    labels = np.array(labels)
    return data[perm][:n], labels[perm][:n]


In [28]:
def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.cpu().numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [29]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/final/final1')

# get some random training images
dataiter = iter(loader_train)
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# show images
#matplotlib_imshow(img_grid, one_channel=True)

# write images to tensorboard
#writer.add_image('cifar10', img_grid)
# write network to tensorflow
cnn_challenge_model = winner(in_channel=3, channel_1=60, channel_2=40, channel_3=40, num_classes=10, alpha=1e-2)
cnn_challenge_optimizer = optimizer = optim.AdamW(cnn_challenge_model.parameters())


writer.add_graph(cnn_challenge_model, images)

"""
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
# select random images and their target indices
images, labels = select_n_random(cifar10_train.data, cifar10_train.targets)

# get the class labels for each image
class_labels = [classes[lab] for lab in labels]
images = np.transpose(images,(0,3,1,2))
print(images.shape)
images = torch.from_numpy(images)
# log embeddings
features = np.reshape(images,(-1, 32 * 32 * 3))
writer.add_embedding(features,
                    metadata=class_labels,
                     label_img=images)
"""

train_part35(cnn_challenge_model, cnn_challenge_optimizer, epochs=20)
print("Training  Done")
writer.close()

Current epoch is:  0
Iteration 0, loss = 2.6821
15.212244897959184 14.799999999999999
Iteration 200, loss = 1.3466
56.18571428571428 53.6
Iteration 400, loss = 1.5643
63.046938775510206 58.4
Iteration 600, loss = 1.0640
68.18367346938776 62.6
Current epoch is:  1
Iteration 0, loss = 0.7189
71.51836734693877 67.0
Iteration 200, loss = 1.1350
71.91224489795918 64.3
Iteration 400, loss = 0.9837
75.04285714285714 67.0
Iteration 600, loss = 1.3076
75.29183673469387 67.0
Current epoch is:  2
Iteration 0, loss = 0.7964
77.02857142857142 67.2
Iteration 200, loss = 0.5706
77.73673469387755 68.4
Iteration 400, loss = 0.7397
80.29591836734694 69.6
Iteration 600, loss = 0.7567
80.95510204081633 69.3
Current epoch is:  3
Iteration 0, loss = 0.4823
80.7469387755102 69.1
Iteration 200, loss = 0.4942
83.27755102040815 69.89999999999999
Iteration 400, loss = 0.7166
83.05918367346938 69.8
Iteration 600, loss = 0.4663
83.7326530612245 71.89999999999999
Current epoch is:  4
Iteration 0, loss = 0.4286
84.3

## Describe what you did 

In the cell below you should write an explanation of what you did, any additional features that you implemented, and/or any graphs that you made in the process of training and evaluating your network.

1. Accuracy and Loss graphs

         From now on I moved to tensorboard.Because torch 1.0 does not support tensorboard, i created a new one with 1.6. So all my operations will be done using pytorch 1.6. I also used a new training function named 'train_part35'
         
         All the runs are in the runs folder, you can load them with tensorboard.


2. Looking for the initial accuracy/loss with default four layer network (1 epoch)
            
   
3. Change the optimizer to AdamW
        
        AdamW is an adam family function, Lets see the result:
            Training: 63
            Val     : 55 

4. Change the activation function
        
        I tried different activation functions but it looks like for 1 epoch they dont really change much,   So I am going to make my architecture definetely bigger and train for more epoch...
        
        Tested Ones: LReLU, PReLu, SeLU
        
5. Change the architecture

        I tried something similar to CIFAR10 winner, 'Scalable Bayesian Optimization Using Deep Neural Networks' for my architecture, its not same but i am not interested with exact architecture but layer size.

        The default training and validation accuracy for this architecture is: (Epoch = 10)
        
            Training:
            Validation: Lower than small network probably hyperparams are wrong
          
        Tested Ones: Architectures/Arch1, Architectures/Arch2

6. Hyperparameter Tunning
        
        We can start a small hyperparameter tuning with this architecture to fit our data fully we should see very good results.
        
        I tried to fit a learning rate but it looks like changing the architecture more than 2-3 layers makes my life harder because fitting the LR is hard, so I am just going to change my architechture slowly.
        
        Tested Ones: LearningRate/Arch2_1e-3
        
7. Change the architecture (But slowly)

       Use smaller filter for first layer,  and multiplied the number of filters by 5  in each layer.

        Tested Ones: Architectures/Arch3,  - Arhitectures/Arch12

        Arch4 overfits, its a very shallow network, but already overfits, that makes me force to decrease the number of filters in each layer and increase the deepness to decrease the overfitting but increasing the accuracy.
        
        Arch5: Max Pool added, Arch6: Two more convolutions (LR needs to be adapted), Arch7: One of the conv removed. 
        
        Decreasing the number of filters and increasing the deepness actually kinda helped me (By just decreasing the accuracy) , but i was still experienced the overfitting so i kinda turned back to a simpler architecture, (Arch12) where I have the accuracy of:
        
        Train:92.78775510204082 
        Val:  71.6
        Test: 70.13
        
        So good news is I already accomplished the homework requriments with this very shallow network. But I can do better...
        
        Lets add Dropout of 0.2 after every conv layer and 0.5 after fc...
        
        It looks like definitely helped with my overfitting. But I also lost some of my accuracy... It looks like the smart thing to do right now is increasing the deepness
  
        I inreased the deepness but LR failed, now lets tune the LR
        
        I spend too much time on fitting the LR but couldnt accomplished anything so i am just going to leave it with my Arch12
  
 

## Test set -- run this only once

Now that we've gotten a result we're happy with, we test our final model on the test set (which you should store in best_model). Think about how this compares to your validation set accuracy.

In [30]:
best_model = cnn_challenge_model
check_accuracy_part34(loader_test, best_model)

Checking accuracy on test set
Got 7006 / 10000 correct (70.06)
